In [1]:
!pip install minisom

  Preparing metadata (setup.py) ... done
  Created wheel for minisom: filename=MiniSom-2.3.2-py3-none-any.whl size=10650 sha256=de3294e1398cb2f103ff3dae0a4ea7c5943b9f0a5a32e16a300f7b92e7efc684
  Stored in directory: /root/.cache/pip/wheels/b4/f6/8a/9daf8831901c3e3805775633404248f10663d1c80b7e5a1314
Successfully built minisom


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from minisom import MiniSom
from sklearn.neighbors import KNeighborsClassifier
import pickle
# Evaluate performance
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier

import time



##urls

In [3]:
# Define the function for normalization using tanh-estimator method, excluding specified columns
def normalize_with_tanh_estimator(data, mean_std_dict, exclude_cols):
    normalized_data = pd.DataFrame()
    for col in data.columns:
        if col not in exclude_cols:
            mu, sigma = mean_std_dict[col]
            normalized_col = 0.5 * (np.tanh(0.1 * ((data[col] - mu) / sigma)) + 1)
            normalized_data[col] = normalized_col
        else:
            normalized_data[col] = data[col]  # Keep excluded columns as they are
    return normalized_data

In [4]:
# Function to calculate distance between input sample V and reference point G
def calculate_distance(v, g):
    return np.linalg.norm(v - g)

In [5]:
# Function to classify input sample as attack or normal and update y_pred array
def som_prediction(sample, d_threshold, sigma, g):
  y_pred = []
  for sam in sample:
    distance_to_g = calculate_distance(sam, g)
    # print("distance_to_g", distance_to_g)
    p_d_greater_than_x = 1 - np.exp(- (distance_to_g / sigma)**2)  # Cumulative distribution function
    is_attack = distance_to_g > d_threshold or p_d_greater_than_x > 0.6  # You can adjust the threshold as needed
    y_pred.append(1 if is_attack else 0)

  return y_pred


In [6]:
def true_vs_predicted(y_true, y_pred):

  # Compute confusion matrix
  cm = confusion_matrix(y_true, y_pred)

  # Extract counts from confusion matrix
  attack_as_normal = cm[1, 0]
  normal_as_attack = cm[0, 1]
  attack_as_attack = cm[1, 1]
  normal_as_normal = cm[0, 0]

  # Print the results
  print("attack (1) but predicted as normal =", attack_as_normal)
  print("normal (0) but predicted as attack =", normal_as_attack)
  print("attack (1) predicted as attack =", attack_as_attack)
  print("normal (0) predicted as normal =", normal_as_normal)

def true_vs_predicted_with_return(y_true, y_pred):

  # Compute confusion matrix
  cm = confusion_matrix(y_true, y_pred)

  # Extract counts from confusion matrix
  try:
    attack_as_normal = cm[1, 0]
  except:
    attack_as_normal = None
  try:
    normal_as_attack = cm[0, 1]
  except:
    normal_as_attack = None
  try:
    attack_as_attack = cm[1, 1]
  except:
    attack_as_attack = None
  try:
    normal_as_normal = cm[0, 0]
  except:
    normal_as_normal = None


  return attack_as_normal, normal_as_attack, attack_as_attack, normal_as_normal

def evaluate_true_and_predicted(test_y, y_pred):

  true_vs_predicted(test_y, y_pred)
  print("\n", classification_report(test_y,y_pred, zero_division=1, output_dict=True))
  print("Accuracy : ",accuracy_score(test_y, y_pred))

  return

def evaluate_true_and_predicted_with_return(com,test_y, y_pred):

  attack_as_normal, normal_as_attack, attack_as_attack, normal_as_normal = true_vs_predicted_with_return(test_y, y_pred)
  # Generate classification report
  precision_recall = classification_report(test_y, y_pred, output_dict=True, zero_division=1)
  precision_ddos = precision_recall['1']  # Assuming '1' represents DDoS class

  fdr = (1 - precision_ddos['precision'])*100
  # Calculate False Detection Rate (FDR)
  print("\n", classification_report(test_y,y_pred, zero_division=1))
  accuracy = (accuracy_score(test_y, y_pred))*100
  print(f"\nAccuracy : {accuracy} % ")
  print(f"False Detection Rate (FDR): {fdr:.2f}%")
  result = [com,accuracy, fdr, attack_as_normal, normal_as_attack, attack_as_attack, normal_as_normal]
  print(result)
  print("\n",com)
  # resultant_columns = ['accuracy', 'attack_as_normal', 'normal_as_attack', 'attack_as_attack', 'normal_as_normal']
  # resultant_df = pd.DataFrame(result,columns=resultant_columns)
  result_df = pd.DataFrame(result)
  # result_df.columns = ['comment', 'accuracy', 'FDR', 'attack_as_normal', 'normal_as_attack', 'attack_as_attack', 'normal_as_normal']
  return result_df


In [7]:

def with_som_and_rf(df, com, bandflag):

  if bandflag == 1:
    # Check if the column exists
    column_to_drop = 'bandwidth'
    if column_to_drop in df.columns:
        # Drop the column
        df.drop(columns=[column_to_drop], inplace=True)
        print(f"\n'{column_to_drop}' column dropped.")
    else:
        print(f"\nColumn '{column_to_drop}' not found in the DataFrame.")


  som_d_threshold = 0.13309999999999966  # Predefined distance threshold
  som_sigma =  0.139099999999999  # Probability threshold


  # Define the columns to exclude from normalization
  exclude_cols = ['label']  # Add more columns here if needed
  # Calculate and store the mean and standard deviation for each dimension, excluding specified columns
  mean_std_dict = {}
  for col in df.columns:
      if col not in exclude_cols:
          mean_std_dict[col] = (df[col].mean(), df[col].std())

  # Normalize the dataset using tanh-estimator method, excluding specified columns
  df_normalized = normalize_with_tanh_estimator(df, mean_std_dict, exclude_cols)
  # df_normalized = df


  # Get trained som model
  g = get_model_som(df_normalized)

  #for test
  test_X = df_normalized.iloc[:, :-1].values
  test_y = df_normalized.iloc[:, -1].values

  X_train, X_test, y_train, y_test = train_test_split(test_X, test_y, test_size=0.3, random_state=42)

  rf_model = get_model_rf(X_train, y_train)

  som_start_time = time.time()
  y_pred_som = som_prediction(test_X, som_d_threshold, som_sigma, g)
  som_end_time = time.time()
  som_time = som_end_time - som_start_time
  print(f"SOM time: {som_time} seconds")

  rf_start_time = time.time()
  y_pred_rf = rf_model.predict(test_X)
  rf_end_time = time.time()
  rf_time = rf_end_time - rf_start_time
  print(f"RF time: {rf_time} seconds")


  combined_start_time = time.time()
  y_pred_som_and_rf = som_and_rf_prediction_from_array(test_X, som_d_threshold, som_sigma, g, rf_model)
  combined_end_time = time.time()
  combined_time = combined_end_time - combined_start_time
  print(f"Combined time: {combined_time} seconds")


  #### evaluation
  print("____"*80)
  print("\n For SOM Model")
  print("--"*40)
  som_values = evaluate_true_and_predicted_with_return(com,test_y, y_pred_som)

  print("\n For RF Model")
  print("--"*40)
  rf_values = evaluate_true_and_predicted_with_return(com,test_y, y_pred_rf)

  print("\n For SOM and RF Model")
  print("--"*40)
  y_pred_som_and_rf_values = evaluate_true_and_predicted_with_return(com,test_y, y_pred_som_and_rf)
  result = [som_values, rf_values, y_pred_som_and_rf_values]
  print("____"*80)
  # print(result)
  # resultant_columns = ['som_values', 'rf_values', 'y_pred_som_and_rf_values']
  # resultant_df = pd.DataFrame(result,columns=resultant_columns)
  return result








In [8]:
def get_details_for_som(test_X, test_y, d_threshold, sigma, g):

  y_pred_som = som_prediction(test_X, d_threshold, sigma, g)

  accuracy = accuracy_score(test_y, y_pred_som)
  attack_as_normal, normal_as_attack, attack_as_attack, normal_as_normal = true_vs_predicted_with_return(test_y, y_pred_som)
  fdr = (1 - attack_as_attack / (attack_as_attack + normal_as_attack)) * 100


  return sigma, d_threshold, accuracy, fdr, attack_as_normal, normal_as_attack, attack_as_attack, normal_as_normal





In [9]:
def som_with_varying_sigma_and_d_threshold(df):

  df = df
  sigma_min = 0.001
  sigma_max = 0.5
  sigma_step = 0.01
  d_threshold_min = 0.001
  d_threshold_max = 0.5
  d_threshold_step = 0.01


  # Generate arrays of varying values for sigma and d_threshold
  sigma_values = np.arange(sigma_min, sigma_max + sigma_step, sigma_step)
  d_threshold_values = np.arange(d_threshold_min, d_threshold_max + d_threshold_step, d_threshold_step)
  max = len(sigma_values)*len(d_threshold_values)
  i = 1
  # Define the columns to exclude from normalization
  exclude_cols = ['label']  # Add more columns here if needed
  # Calculate and store the mean and standard deviation for each dimension, excluding specified columns
  mean_std_dict = {}
  for col in df.columns:
      if col not in exclude_cols:
          mean_std_dict[col] = (df[col].mean(), df[col].std())

  # Normalize the dataset using tanh-estimator method, excluding specified columns
  df_normalized = normalize_with_tanh_estimator(df, mean_std_dict, exclude_cols)
  # df_normalized = df


  # Get trained som model
  g = get_model_som(df_normalized)

  #for test
  test_X = df_normalized.iloc[:, :-1].values
  test_y = df_normalized.iloc[:, -1].values

  result = []
  for sigma in sigma_values:
    for d_threshold in d_threshold_values:
      print(f"\r{i/max * 100}---  {max}")
      result.append(get_details_for_som(test_X, test_y, d_threshold, sigma, g))
      i=i+1

  resultant_columns = ['sigma', 'd_threshold', 'accuracy', 'FDR', 'attack_as_normal', 'normal_as_attack', 'attack_as_attack', 'normal_as_normal']
  resultant_df = pd.DataFrame(result,columns=resultant_columns)
  return resultant_df



In [10]:
def avg_time_per_prediction(dataset_size, total_time):
  return(total_time/dataset_size * 1000)

In [11]:
def get_model_som(df_normalized):

  # Filter rows where label equals 0
  som_filtered_df = df_normalized[df_normalized['label'] == 0]

  som_train_X = som_filtered_df.iloc[:, :-1].values
  som_size = 30  # Example size, adjust based on your dataset
  som = MiniSom(som_size, som_size, som_train_X.shape[1], sigma=15, learning_rate=0.1, random_seed=0 )
  som.random_weights_init(som_train_X)
  som.train_batch(som_train_X, 1000, verbose = "true")
  # Calculate the reference point G
  g = np.median(som.get_weights(), axis=(0, 1))

  return g

def get_model_rf(X_train, y_train):
  # Initialize Random Forest model
  rf_model = RandomForestClassifier(random_state=42)

  # Fit the model
  rf_model.fit(X_train, y_train)

  return rf_model


def som_and_rf_prediction_from_array(test_X, d_threshold, sigma, g, rf_model):
  y_result = []
  rf_onlytime = 0
  for sam in test_X:
    distance_to_g = calculate_distance(sam, g)
    p_d_greater_than_x = 1 - np.exp(- (distance_to_g / sigma)**2)  # Cumulative distribution function
    is_attack = distance_to_g > d_threshold or p_d_greater_than_x > 0.6  # You can adjust the threshold as needed
    som_predicted = [1] if is_attack else [0]
    if som_predicted == [0]:
      y_result.append(som_predicted)
    else:

      rf_single_pred_start = time.time()
      rf_pred = rf_model.predict(sam.reshape(1, -1))
      rf_single_pred_end = time.time()
      rf_single_pred_time = rf_single_pred_end - rf_single_pred_start
      rf_onlytime = rf_onlytime + rf_single_pred_time
      y_result.append(rf_pred)
  print("rf_total", rf_onlytime)
  return y_result

import numpy as np
import time

def som_and_rf_prediction_from_array_optimized(test_X, d_threshold, sigma, g, rf_model):
    y_result = []
    rf_onlytime = 0
    distance_to_g_all = np.array([calculate_distance(sam, g) for sam in test_X])
    p_d_greater_than_x_all = 1 - np.exp(- (distance_to_g_all / sigma)**2)

    for i, sam in enumerate(test_X):
        distance_to_g = distance_to_g_all[i]
        p_d_greater_than_x = p_d_greater_than_x_all[i]

        is_attack = distance_to_g > d_threshold or p_d_greater_than_x > 0.6
        som_predicted = [1] if is_attack else [0]

        if som_predicted == [0]:
            y_result.append(som_predicted)
        else:
            rf_single_pred_start = time.time()
            rf_pred = rf_model.predict(sam.reshape(1, -1))
            rf_single_pred_end = time.time()
            rf_single_pred_time = rf_single_pred_end - rf_single_pred_start
            rf_onlytime += rf_single_pred_time
            y_result.append(rf_pred)

    print("rf_total", rf_onlytime)
    return y_result



def som_and_rf_prediction_for_single(sam, d_threshold, sigma, g, rf_model):
    # start_time = time.time()
    # som_predicted = som_prediction([sam], d_threshold, sigma, g)
    distance_to_g = calculate_distance(sam, g)
    p_d_greater_than_x = 1 - np.exp(- (distance_to_g / sigma)**2)  # Cumulative distribution function
    is_attack = distance_to_g > d_threshold or p_d_greater_than_x > 0.6  # You can adjust the threshold as needed
    som_predicted = [1] if is_attack else [0]

    if som_predicted == [0]:
      return som_predicted
    else:
      rf_single_pred_start = time.time()
      rf_pred = rf_model.predict(sam.reshape(1, -1))
      rf_single_pred_end = time.time()
      rf_single_pred_time = rf_single_pred_end - rf_single_pred_start
      # print(f"RF single pred time: {rf_single_pred_time} seconds")
      # rf_pred = [1]
      return rf_pred

      # end_time = time.time()
      # som_time = end_time - start_time
      # print(f"SOM time: {som_time:.2f} seconds")

def rf_model_prediction_for_single(test_X, rf_model):
  y_result = []
  for sam in test_X:
    rf_pred = rf_model.predict(sam.reshape(1, -1))
    y_result.append(rf_pred)
  return y_result


# Load dataset

In [12]:
# Load the dataset

urls = []
comments = []

# 1s split
urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/combined%20Dataset.csv")
comments.append("1s split caida2007")

# #self made data
urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/som_data.csv")
comments.append("self made data")

urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/1ssplit%20combined%20dataset%20CAIDA%202007.csv")
comments.append("1s split with ")

# new 2s splits
#filled
urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/caida2007/csv/2ssplit%20combined%20dataset%20CAIDA%202007%20filled.csv")
comments.append("filled 2s split with bandwidth caida 2007")

#unfilled
urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/caida2007/csv/2ssplit%20combined%20dataset%20CAIDA%202007%20unfilled.csv")
comments.append("unfilled 2s split with bandwidth")

#filled
urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/caida2007/csv/2ssplit%20combined%20dataset%20CAIDA%202007%20filled.csv")
comments.append("filled 2s split without bandwidth")

# unfilled
urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/caida2007/csv/2ssplit%20combined%20dataset%20CAIDA%202007%20unfilled.csv")
comments.append("unfilled 2s split without bandwidth")

## ids 2017 benign only
#filled
urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/isd%202017/ids-2017-benign-Monday-WorkingHours2s%20unlabeled_filled_entropy.csv")
comments.append("ids 2017 benign only filled")

urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/isd%202017/ids-2017-benign-Monday-WorkingHours2s%20unlabeled_filled_entropy.csv")
comments.append("ids 2017 benign only filled without bandwidth")

##unfilled
urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/isd%202017/ids-2017-benign-Monday-WorkingHours2s%20unlabeled_unfilled_entropy.csv")
comments.append("ids 2017 benign only unfilled")

urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/isd%202017/ids-2017-benign-Monday-WorkingHours2s%20unlabeled_unfilled_entropy.csv")
comments.append("ids 2017 benign only unfilled without bandwidth")


In [13]:

def master(df, com, bandflag):
  #for  evaluating time of single evaluation

  if bandflag == 1:
    # Check if the column exists
    column_to_drop = 'bandwidth'
    if column_to_drop in df.columns:
        # Drop the column
        df.drop(columns=[column_to_drop], inplace=True)
        print(f"\n'{column_to_drop}' column dropped.")
    else:
        print(f"\nColumn '{column_to_drop}' not found in the DataFrame.")


  som_d_threshold = 0.13309999999999966  # Predefined distance threshold
  som_sigma =  0.139099999999999  # Probability threshold


  # Define the columns to exclude from normalization
  exclude_cols = ['label']  # Add more columns here if needed
  # Calculate and store the mean and standard deviation for each dimension, excluding specified columns
  mean_std_dict = {}
  for col in df.columns:
      if col not in exclude_cols:
          mean_std_dict[col] = (df[col].mean(), df[col].std())

  # Normalize the dataset using tanh-estimator method, excluding specified columns
  df_normalized = normalize_with_tanh_estimator(df, mean_std_dict, exclude_cols)
  # df_normalized = df


  # Get trained som model
  g = get_model_som(df_normalized)

  #for test
  test_X = df_normalized.iloc[:, :-1].values
  test_y = df_normalized.iloc[:, -1].values

  X_train, X_test, y_train, y_test = train_test_split(test_X, test_y, test_size=0.3, random_state=42)

  rf_model = get_model_rf(X_train, y_train)




  data_size = len(test_X)
  som_start_time = time.time()
  y_pred_som = som_prediction(test_X, som_d_threshold, som_sigma, g)
  som_end_time = time.time()
  som_time = som_end_time - som_start_time
  print(f"\nSOM Avg per sample : {avg_time_per_prediction(data_size, som_time)} ms")
  print(f"SOM time: {som_time} seconds\n")

  rf_start_time = time.time()
  # y_pred_rf = rf_model.predict(test_X)
  y_pred_rf = rf_model_prediction_for_single(test_X, rf_model)
  rf_end_time = time.time()
  rf_time = rf_end_time - rf_start_time
  print(f"RF Avg per sample : {avg_time_per_prediction(data_size, rf_time)} ms")
  print(f"RF time: {rf_time} seconds\n")


  combined_start_time = time.time()
  y_pred_som_and_rf = som_and_rf_prediction_from_array_optimized(test_X, som_d_threshold, som_sigma, g, rf_model)
  combined_end_time = time.time()
  combined_time = combined_end_time - combined_start_time
  print(f"Combined Avg per sample : {avg_time_per_prediction(data_size, combined_time)} ms")
  print(f"Combined time: {combined_time} seconds\n")

  combined_start_time2 = time.time()
  y_pred_som_and_rf2 = som_and_rf_prediction_from_array(test_X, som_d_threshold, som_sigma, g, rf_model)
  combined_end_time2 = time.time()
  combined_time2 = combined_end_time2 - combined_start_time2
  print(f"Combined 2 Avg per sample : {avg_time_per_prediction(data_size, combined_time2)} ms")
  print(f"Combined 2 time: {combined_time2} seconds\n")



  #### evaluation
  print("____"*80)
  print("\n For SOM Model")
  print("--"*40)
  som_values = [evaluate_true_and_predicted_with_return(com,test_y, y_pred_som), som_time, avg_time_per_prediction(data_size, som_time)]

  print("\n For RF Model")
  print("--"*40)
  rf_values = [evaluate_true_and_predicted_with_return(com,test_y, y_pred_rf), rf_time, avg_time_per_prediction(data_size, rf_time)]

  print("\n For SOM and RF Model")
  print("--"*40)
  y_pred_som_and_rf_values = [evaluate_true_and_predicted_with_return(com,test_y, y_pred_som_and_rf), combined_time, avg_time_per_prediction(data_size, combined_time)]
  result = [som_values, rf_values, y_pred_som_and_rf_values]
  print("____"*80)
  # print(result)
  # resultant_columns = ['som_values', 'rf_values', 'y_pred_som_and_rf_values']
  # resultant_df = pd.DataFrame(result,columns=resultant_columns)
  return result








In [14]:


# #self made data
# urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/som_data.csv")
# comments.append("self made data")





results = []
iteration = 0
for url in urls:
  comment = comments[iteration]
  if 'without bandwidth' in comment:
    bandflag = 1
  else:
    bandflag = 0
  print(f"Iteration : {iteration}")

  df = pd.read_csv(url)

  # y_pred_som_and_rf = with_som_and_rf(df,comment, bandflag)
  value = master(df,comment, bandflag)
  results.append(value)
  iteration = iteration + 1


Iteration : 0
 [ 1000 / 1000 ] 100% - 0:00:00 left 
 quantization error: 0.006262405267559605

SOM Avg per sample : 0.010112533994371845 ms
SOM time: 0.007260799407958984 seconds

RF Avg per sample : 13.312394572499736 ms
RF time: 9.55829930305481 seconds

rf_total 6.622119665145874
Combined Avg per sample : 9.271993278460915 ms
Combined time: 6.6572911739349365 seconds

rf_total 6.65443754196167
Combined 2 Avg per sample : 9.347583922170996 ms
Combined 2 time: 6.711565256118774 seconds

________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

 For SOM Model
--------------------------------------------------------------------------------

               precision    recall  f1-score   support

           0       1.00      0.8

In [15]:
# result_som_with_varying_sigma_and_d_threshold = som_with_varying_sigma_and_d_threshold(df)


In [16]:
value_pd = pd.DataFrame(value)
value_pd

,0,1,2
0,...,0.106841,0.007333
1,...,102.264529,7.019324
2,...,24.011821,1.648145


In [17]:
# y_pred_som_and_rf
value_1 = pd.DataFrame(value_pd[0])
value_1

,0
0,...
1,...
2,...


In [18]:
# results

In [19]:
# resultant_columns = ['som_values', 'rf_values', 'y_pred_som_and_rf_values']
# resultant_df = pd.DataFrame(results,columns=resultant_columns)

In [20]:
# resultant_df_som = resultant_df['som_values']
# resultant_df_rf = resultant_df['rf_values']
# resultant_df_som_and_rf = resultant_df['y_pred_som_and_rf_values']

In [21]:
# resultant_df_som

In [22]:
# resultant_df_rf

In [23]:
# resultant_columns = ['com','accuracy', 'attack_as_normal', 'normal_as_attack', 'attack_as_attack', 'normal_as_normal']
# resultant_df_som = pd.DataFrame(resultant_df_som,columns=resultant_columns)

In [24]:
# resultant_df_som

In [25]:
# y_pred_som_and_rf

In [26]:
# result